In [ ]:
import torch
import open_clip
import pandas as pd

from src.fairface import get_img_name
from src.utils import load_json
from src.encoder import CLIPEncoder

In [2]:
backbone = "ViT-L-14"
datasource = "laion2b_s32b_b82k"
model, _, preprocess = open_clip.create_model_and_transforms(backbone, pretrained=datasource)
model.eval()
tokenizer = open_clip.get_tokenizer(backbone)

/home/lucasmc/anaconda3/envs/clipenv/lib/python3.10/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_

In [ ]:
label_dict = load_json("../labels/raw_gender_labels.json")
fface = "/home/lucasmc/Documents/ufrgs/data/datasets/FairFace/"
split = "val"
fface_csv = f"{fface}/fface_{split}.csv"
embs_path = f"{fface}/embeddings/{split}/{backbone}/{datasource}"

In [4]:
fface_df = pd.read_csv(fface_csv)
files = fface_df["file"]
files[0]

'val/1.jpg'

In [5]:
label_dict

{'Male': 'a photo of a man', 'Female': 'a photo of a woman'}

In [9]:
labels = list(label_dict)
prompts = list(label_dict.values())
text = tokenizer(prompts)
preds_dict = {
    "file": [],
    "gender_preds": []
}

for file in files:
    img_name = get_img_name(file)
    image_features = CLIPEncoder.load_embeddings(f"{embs_path}/{img_name}.npy")
    image_features = image_features.to(device="cuda")
    with torch.no_grad(), torch.autocast("cuda"):
        text_features = model.encode_text(text).to(device="cuda")
        text_features /= text_features.norm(dim=-1, keepdim=True)
        
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    winner = text_probs.argmax()
    preds_dict['file'].append(file)
    preds_dict['gender_preds'].append(labels[winner])


/home/lucasmc/Documents/ufrgs/clip-face-attribs/src/encoder.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(emb_path)


In [18]:
preds_df = pd.DataFrame(preds_dict)

In [22]:
result_df = pd.merge(fface_df, preds_df, on="file")
result_df.head()

,file,age,gender,race,service_test,gender_preds
0,val/1.jpg,3-9,Male,East Asian,False,Male
1,val/2.jpg,50-59,Female,East Asian,True,Female
2,val/3.jpg,30-39,Male,White,True,Male
3,val/4.jpg,20-29,Female,Latino_Hispanic,True,Female
4,val/5.jpg,20-29,Male,Southeast Asian,False,Male


In [30]:
result_df.set_index('file')

,age,gender,race,gender_preds
file,,,,
val/1.jpg,3-9,Male,East Asian,Male
val/2.jpg,50-59,Female,East Asian,Female
val/3.jpg,30-39,Male,White,Male
val/4.jpg,20-29,Female,Latino_Hispanic,Female
val/5.jpg,20-29,Male,Southeast Asian,Male
...,...,...,...,...
val/10950.jpg,30-39,Male,White,Male
val/10951.jpg,50-59,Male,White,Male
val/10952.jpg,60-69,Male,Latino_Hispanic,Male


In [ ]:
result_df.drop(columns=['service_test'], inplace=True)

In [31]:
out_f = f"./results/raw_gender_{backbone}_{datasource}_{split}.csv"
result_df.to_csv(out_f, index=False)